# Note !

We can map these short description to its corresponding full description, as EG to be Egypt.

In [1]:
# Main libraries 
from sklearn.preprocessing import LabelEncoder
from fastapi.encoders import jsonable_encoder
from pydantic import BaseModel
from fastapi import FastAPI
from enum import Enum
import pandas as pd
import nest_asyncio
import uvicorn
import os
import io
import numpy as np
# Our files
from data_shuffling_split import *
from features_extraction import *
from data_preprocess import *
from ml_modeling import *
from keras_models import *
from configs import *

# Note !

Based on what we got from **Compare ML Models**, we end up by decide of using *Abo Bakr Word2vec model*, alongside with AdaBoost Classifier.

**We need to map the label as it numbers into corresponding class delicate do we use LabelEncoder as before**

In [2]:
number_of_features = 100
max_len_str  = 64
word2vec_path = "bakrianoo_unigram_cbow_model/full_uni_cbow_100_twitter.mdl"
word_to_vec_model = load_word2vec_model("models/word2vec/" + word2vec_path)

strat_test_set = pd.read_csv("dataset/test/strat_test_set.csv")
l_encoder = LabelEncoder()
l_encoder.fit(list(strat_test_set["dialect"]))
l_encoder.classes_

array(['AE', 'BH', 'DZ', 'EG', 'IQ', 'JO', 'KW', 'LB', 'LY', 'MA', 'OM',
       'PL', 'QA', 'SA', 'SD', 'SY', 'TN', 'YE'], dtype='<U2')

In [3]:
def classify_text(text, choosed_model, word_to_vec_model=word_to_vec_model, 
                  max_len_str=max_len_str, l_encoder=l_encoder):
    '''
    The function used to predict the tweets requested by the API.
    
    Argumen
        text              : string, the text we need to classify.
        word_to_vec_model : word2vec object, our word embedding model.
        max_len_str       : integer, The maximum number of tokens represent each text.
        l_encoder         : object, the encode of each class into some label to retrieve the class from.
    Return
        classifed_text    : dict, the returned classified text into the api request.
    '''
    # First process the text requested
    text_cleaned = clean_text(text)
    # Second tokenize that text to get the representation of each token
    tokenized_text = tokenize_using_nltk_TreebankWordTokenizer([text_cleaned])
    # Retrieve the whole text representation
    text_features = text_to_matrix_using_word2vec(word_to_vec_model, tokenized_text, max_len_str)
    
    # No prediction yet
    predicted = ''
    # Prediction with chosen ML Model
    if choosed_model == "Machine Learning Model":
        print("=====================Machine Model=====================")
        model_path    = "bakr/AdaBoostClassifier__f1_0.325_ml.sav"
        cls_model     = pickle_load_model("models/ml_models/" + model_path)
        
        # it return a list with one index to retrieve it directly
        predicted = cls_model.predict(text_features)
    elif choosed_model == "Deep Learning Model":
        print("=====================Deep Model=====================")
        model_path    = "models/dl_models/run_with_bakr_word2vec_Adam_lstm_with_batch_learning_rate=0.1__model.h5"
        text_features = text_features.reshape(text_features.shape[0], max_len_str, number_of_features)
        dl_model      = keras_load_model(model_path)
        predicted     = dl_model.predict(text_features)
        predicted     =np.argmax(predicted,axis=1)
        
    
    pred_result = l_encoder.inverse_transform(predicted)
    # Return dictionary with request text and the predicted class
    classifed_text = {
        'query': text,
        'We have predict your dialect text as: ': str(pred_result[0])
    }
    return classifed_text

In [4]:
# Assign an instance of the FastAPI class to the variable "app" to interact with the api.
app = FastAPI(title='Deploying a ML & DL Model with FastAPI')

# List available models using Enum
class Model(str, Enum):
    """The class to choose from the models we have, Enum represent fixed value that can not change"""
    ML_Model = "Machine Learning Model"
    DL_Model = "Deep Learning Model"


class Text(BaseModel):
    """
    The class used to get the text from the user, 
    """
    user_attention: str = """Put your Arabic text in empty value of key "query" below to predict it and choose which model architecture you need."""
    query         : str = ''
        

@app.get("/")
def home():
    # Once you go to this link you will see the get and post method below to trying out
    return "Congratulations! Your API is working as expected. Now head over to http://localhost:8000/docs."


# This endpoint handles all the logic necessary for the object detection to work.
# It requires the desired model and the dictionary of tweet and default class as we give default values to us
# In the api you can try other tweet from some_tweets below
@app.post("/predict") 
def prediction(model: Model, text_dict: Text):
    
    # Encode the retrived request data 
    text_dict      = jsonable_encoder(text_dict)
    # Run our model
    try:
        classifed_text = classify_text(text_dict['query'], model)
    except:
        classifed_text = {
        'Error': "Please Provide us with Arabic text in empty value of key 'query' in the Request body, Thanks ^^."
        }
    return classifed_text

In [ ]:
# Allows the server to be run in this interactive environment
nest_asyncio.apply()

# Host depends on the setup you selected
host = "0.0.0.0"

# uvicorn is fast Asynchronous Server Gateway Interface (ASGI) uvicorn handles the serving
# Spin up the server!    
uvicorn.run(app, host=host, port=8000)

INFO:     Started server process [5502]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:42596 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:42596 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:42596 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:42596 - "GET /openapi.json HTTP/1.1" 200 OK
